In [2]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import models

# -----------------------------
# Dataset Loader
# -----------------------------
class IDRiDDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image, torch.tensor(label, dtype=torch.long), None

# -----------------------------
# Model Definition
# -----------------------------
class ClassificationModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# -----------------------------
# Training Function
# -----------------------------
def train(model, dataloader, optimizer, epoch, device, writer):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch_idx, (images, labels, _) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        writer.add_scalar('Classification/Train_Loss', loss.item(), epoch * len(dataloader) + batch_idx)
        writer.add_scalar('Classification/Train_Accuracy', correct / total, epoch * len(dataloader) + batch_idx)
    return total_loss / len(dataloader), correct / total

# -----------------------------
# Evaluation Function
# -----------------------------
def evaluate(model, dataloader, device, epoch, writer):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch_idx, (images, labels, _) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            writer.add_scalar('Classification/Val_Loss', loss.item(), epoch * len(dataloader) + batch_idx)
            writer.add_scalar('Classification/Val_Accuracy', correct / total, epoch * len(dataloader) + batch_idx)
    return total_loss / len(dataloader), correct / total

# -----------------------------
# Main Execution
# -----------------------------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    writer = SummaryWriter(log_dir="runs/classification_training")

    # Paths (same as original code)
    image_dir = r"D:\Project_Space\5.retina\retina_data\B. Disease Grading\1. Original Images\a. Training Set"
    label_csv = r"D:\Project_Space\5.retina\retina_data\B. Disease Grading\2. Groundtruths\a. IDRiD_Disease Grading_Training Labels.csv"
    val_image_dir = r"D:\Project_Space\5.retina\retina_data\B. Disease Grading\1. Original Images\b. Testing Set"
    val_label_csv = r"D:\Project_Space\5.retina\retina_data\B. Disease Grading\2. Groundtruths\b. IDRiD_Disease Grading_Testing Labels.csv"

    image_paths = sorted(glob.glob(image_dir + r"\*.jpg"))
    label_df = pd.read_csv(label_csv)
    filename_to_label = dict(zip(label_df['Image name'], label_df['Retinopathy grade']))
    labels = [filename_to_label[os.path.splitext(os.path.basename(p))[0]] for p in image_paths]

    val_image_paths = sorted(glob.glob(val_image_dir + r"\*.jpg"))
    val_label_df = pd.read_csv(val_label_csv)
    val_filename_to_label = dict(zip(val_label_df['Image name'], val_label_df['Retinopathy grade']))
    val_labels = [val_filename_to_label[os.path.splitext(os.path.basename(p))[0]] for p in val_image_paths]

    # Transforms
    transform = A.Compose([
        A.Resize(256, 256),
        A.Normalize(mean=(0.5,), std=(0.5,)),
        ToTensorV2()
    ])

    # Datasets and Loaders
    train_dataset = IDRiDDataset(image_paths, labels, transform=transform)
    val_dataset = IDRiDDataset(val_image_paths, val_labels, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

    # Model
    model = ClassificationModel(num_classes=5).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Training Loop
    for epoch in range(10):
        train_loss, train_acc = train(model, train_loader, optimizer, epoch, device, writer)
        print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
        val_loss, val_acc = evaluate(model, val_loader, device, epoch, writer)
        print(f"Epoch {epoch+1} - Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

    # Save Model
    os.makedirs("saved_models", exist_ok=True)
    torch.save(model.state_dict(), "saved_models/classification_model.pth")
    print("Model saved to saved_models/classification_model.pth")

    writer.close()

if __name__ == "__main__":
    main()


d:\Project_Space\5.retina\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Project_Space\5.retina\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>